# CSM Simulation Groupwork Project

A bank operates with a single service counter. Customers arrive randomly and are served on a first-come, first-served basis. You are required to simulate this system using Python.

## Assumptions
- Inter-arrival times of customers are uniformly distributed between 1 and 8 minutes.
- Service times are uniformly distributed between 1 and 6 minutes.
- There is only one server (single-server queue).
- The queuing discipline is FIFO (First-In, First-Out).

## Task
Your group is expected to:
1. Generate inter-arrival and service times for 500 customers using the specified uniform
distributions.
2. Implement a Python simulation of the system using a discrete-event simulation approach.
3. Compute and report performance metrics.
4. Produce relevant visualizations.

## Setup
Create an environment and activate by running
 ```bash
 python -m venv venv
 venv\Scripts\activate
 pip install ipykernel
 ```

Then install the packages by running the command below:

In [81]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Code Implementation

Import the libraries

In [82]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt

Create an array to store the arrival, service, service start, service end and wait times

And arrays for idle time calculation

In [83]:
simulation_data = [] # Store simulation data for each customer
last_service_end = [0] # List to store service end time for the last customer
total_idle_time = [0] # List to store total idle time for the server when new customer arrives

Created a function to simulate each individual customer

In [84]:
def customer(env, name, counter, inter_arrival_time):
    arrival_time = round(env.now, 2)

    with counter.request() as req:
        yield req
        service_start_time = round(env.now, 2)
        wait_time = round(env.now - arrival_time, 2)

        # Calculate idle time if any
        if service_start_time > last_service_end[0]:
            idle = round(service_start_time - last_service_end[0], 2)
            total_idle_time[0] += idle
        else:
            idle = 0

        service_time = round(random.uniform(1, 6), 2)
        yield env.timeout(service_time)

        service_end_time = round(env.now, 2)
        time_in_system = round(service_end_time - arrival_time, 2)

        # Update last_service_end
        last_service_end[0] = service_end_time

        simulation_data.append({
            'Customer': name,
            'Inter-Arrival Time': inter_arrival_time,
            'Arrival Time': arrival_time,
            'Wait Time': wait_time,
            'Service Time': service_time,
            'Service Start Time': service_start_time,
            'Service End Time': service_end_time,
            'Time in System': time_in_system,
            'Idle Time Before Service': idle
        })

Creating a function for Inter-Arrival Time (IAT) for a specific number of customers

In [85]:
max_customers = 500

In [86]:
def IAT(env, counter):
    current_customer = 0
    while current_customer < max_customers:
        # Generate inter-arrival time uniformly between 1 and 8 minutes
        inter_arrival_time = random.uniform(1, 8)
        # print(f'Inter-arrival time for customer {current_customer + 1}: {inter_arrival_time:.2f} minutes')
        yield env.timeout(inter_arrival_time)
        
        # Create a new customer
        current_customer += 1
        
        # Print and process the arrival of the customer
        # print(f'{customer} arrives at {env.now:.2f} minutes')
        env.process(customer(env, current_customer, counter, round(inter_arrival_time, 2)))

Setup environment and resource

In [87]:
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)  # Single server since we are doing MM1

Start arrival process

In [88]:
env.process(IAT(env, counter))  # Start the inter-arrival time process
env.run()

## Simulation Table

Printing the simulation table using a dataframe

In [90]:
df = pd.DataFrame(simulation_data)
df.to_csv('Simulation_Results.csv', index=False)
df.head(10)

,Customer,Inter-Arrival Time,Arrival Time,Wait Time,Service Time,Service Start Time,Service End Time,Time in System,Idle Time Before Service
0,1,3.32,3.32,-0.00,5.90,3.32,9.22,5.90,3.32
1,2,5.59,8.91,0.31,3.40,9.22,12.62,3.71,0.00
2,3,4.16,13.08,-0.00,1.37,13.08,14.45,1.37,0.46
3,4,4.52,17.59,0.00,1.06,17.59,18.65,1.06,3.14
4,5,8.00,25.59,0.00,3.53,25.59,29.12,3.53,6.94
5,6,6.28,31.87,-0.00,3.59,31.87,35.46,3.59,2.75
6,7,4.29,36.16,0.00,2.83,36.16,38.99,2.83,0.70
7,8,1.13,37.30,1.69,1.04,38.99,40.03,2.73,0.00
8,9,7.93,45.22,0.00,1.29,45.22,46.51,1.29,5.19
9,10,1.01,46.23,0.28,2.26,46.51,48.77,2.54,0.00


## Visualizations